In [ ]:
import pandas as pd 
import numpy as np
import pandas_market_calendars as mcal
from os import listdir
from os.path import isfile, join

import pytz
eastern = pytz.timezone('US/Eastern')

# Daily Data

## Import Stocks

In [ ]:
stocks = pd.read_pickle("data/stocks.pkl")

In [ ]:
stocks.head(3)

In [ ]:
stock_tickers = set(stocks.index.get_level_values("ID").unique())
len(stock_tickers)
#test

## Import Stories

In [ ]:
#stories = pd.read_pickle("data/stories.pkl")
stories = pd.read_parquet("data/raw_bzg/story_df_raw_2019.parquet")

In [ ]:
stories_tickers = set(stories.stocks.unique())

## Parse stocks

In [ ]:
assert stories.stocks.dtype == stocks.index.dtypes[1]

In [ ]:
def add_targets(df):
    required_columns = ["Close", "High", "Low", "Open"]
    # df.loc[:, "IntradayReturn"] = df["Close"]/df["Open"] - 1
    df.loc[:, "CloseToCloseReturn"] = df["Close"] / df.shift(1)["Close"] - 1
    # df.loc[:, "NextDayReturn"] = df.shift(-1)["Close"] / df.shift(-1)["Open"] - 1
    # df.loc[:, "CloseToNextOpen"] = df.shift(-1)["Open"] / df["Close"] - 1
    return df

In [ ]:
stocks.index.dtypes

In [ ]:
stocks.loc[:, ["IntradayReturn", "NextDayReturn"]] = np.nan
stocks = stocks.swaplevel(0, 1).sort_index(ascending=[True, True])

In [ ]:
stocks = stocks.groupby("ID", as_index=False).apply(add_targets)
stocks.index = stocks.index.droplevel(None)

## Parse Stories

In [ ]:
from pandas.tseries.offsets import BDay

In [ ]:
# PARAMETER 
typ = "CloseToCloseReturn"

In [ ]:
def get_appropriate_date(timestamp, typ):
    if typ == "CloseToCloseReturn":
        # TODO: Some noise here due to closing auction?
        if timestamp.hour < 16: return timestamp.date()
        if timestamp.hour >= 16: return timestamp.date() +  BDay(1)

In [ ]:
# test
get_appropriate_date(stories.NewsTimestamp.iloc[4], typ="CloseToCloseReturn")

### Date assignment

In [ ]:
# If we use Intraday return then news should only be between 9:40 am and 4pm (us trading hours).
# If we use close-to-close return then news for this days CTC should be between yesterday 4pm and today 4pm. 
stories.loc[:, "Date"] = stories.NewsTimestamp.apply(lambda x: get_appropriate_date(x, typ))
stories = stories.astype({"Date":'datetime64[ns]'})

## Merging

In [ ]:
stories.rename(columns=dict(stocks="ID"), inplace=True)

In [ ]:
stories.columns

In [ ]:
stocks.columns

In [ ]:
dataset = stories[["Date", "NewsTimestamp", "ID", "body"]].\
    merge(stocks[
        [
            # "IntradayReturn", 
            # "NextDayReturn", 
            "CloseToCloseReturn"]
         ], on=["Date", "ID"], how="inner")

In [ ]:
dataset[dataset.isna().sum(axis=1) > 0]

In [ ]:
dataset = dataset.dropna()

	Date	NewsTimestamp	ID	body	CloseToCloseReturn

In [ ]:
dataset.to_pickle("data/dataset.pkl")

## Create train-test-split 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
dataset = pd.read_pickle("data/dataset.pkl")
assert dataset.index.is_unique

In [ ]:
test_size = 0.2
seed = 420
### Train-test split -> Auslagern
train_idx, test_idx = train_test_split(dataset.index, test_size=0.2, random_state=seed)

In [ ]:
test_idx

## Filter training set for general stock market events

In [ ]:
# Select S&P
SnP = stocks.query("ID == 'A0AET0'")

In [ ]:
SnP = add_targets(SnP)

In [ ]:
alpha = 0.1 # Percentage observations classified as too extreme to be used in the training set  

In [ ]:
target_col = "CloseToCloseReturn"

In [ ]:
lower_quantile = SnP.loc[:, target_col].quantile(alpha/2)

In [ ]:
upper_quantile = SnP.loc[:, target_col].quantile(1-alpha/2)

In [ ]:
print(f"Upper Quantile: {upper_quantile:.4f}. Lower Quantile: {lower_quantile:.4f}")

In [ ]:
mask = (SnP.loc[:, target_col] >= lower_quantile) & (SnP.loc[:, target_col] <= upper_quantile)

# Only select dates where SnP behaved calmly
allowed_dates = SnP.loc[mask, :].index.get_level_values("Date")

In [ ]:
# Now trim  training set
train_dat = dataset.loc[train_idx, :]
adj_train_idx = train_dat.loc[train_dat.Date.isin(allowed_dates)].index

In [ ]:
train_idx

In [ ]:
adj_train_idx

## Save train and test indices

In [ ]:
with open('data/dataset_train_test_idx.pkl', 'wb') as f:
    pickle.dump((adj_train_idx, test_idx), f)